### Edgar Moises Hernandez-Gonzalez
#### 06/06/20-09/06/20
#### Clasificacion de imagenes motoras en EEG con CNN-2D
##### Caracteristicas = Espectrogramas STFFT (3D)
##### Concatenacion vertical
## Resize proporcional de 264x8
##### 2 seg

In [1]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from scipy.signal import spectrogram
import pywt
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

Using TensorFlow backend.


In [ ]:
# para que este script funcione debe de decir 'channels_last', si dice 'channels_first' no sirve
K.image_data_format()

'channels_last'

In [ ]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/MI-EEG-A01T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/MI-EEG-A01E.csv", header=None)
y_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/EtiquetasA01T.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/EtiquetasA01E.csv", header=None)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(288, 22000)
(288, 22000)
(288, 1)
(288, 1)


In [ ]:
x = pd.concat([x_train, x_test])
y = pd.concat([y_train, y_test])

In [ ]:
print(x.shape)
print(y.shape)

(576, 22000)
(576, 1)


In [ ]:
# en caso de no tener el y_test se puede utilizar el x_test
n_samples = len(y)

print("n_samples:", n_samples)

n_samples: 576


In [ ]:
# esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test
n_classes = len(np.unique(y))

print("n_classes:", n_classes)

n_classes: 4


In [ ]:
# codificar etiquetas con valores de 0 a nClases-1
y = LabelEncoder().fit_transform(y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#unir 22 canales
def unir_escalogramas_vertical_resize(data):
  escalas = np.arange(1,113)
  dim = (62, 264) #ancho, alto
  datos = np.zeros((576,264,62))
  temporal = np.zeros((2112,500))
  for i in range(data.shape[0]): #288 muestras
    for j in range(22): #22 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      #sig = sig[124:624]
      coef, freqs = pywt.cwt(sig, escalas, 'cmor3-3') #escalograma CWT
      x = abs(coef)
      temporal[j*96:(j+1)*96, :] = x[-96:, :]
    resized = cv2.resize(temporal, dim, interpolation=cv2.INTER_AREA)
    datos[i] = resized
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [ ]:
# unir_escalogramas_vertical_resize(data):
inicio = time.time()

x = unir_escalogramas_vertical_resize(x)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
300
400
500
Tiempo: 393.5382490158081


In [ ]:
print(x.shape)

(576, 264, 62)


In [ ]:
print(np.max(x))
print(np.min(x))
print(np.mean(x))

49.663303817366454
0.039463022631068434
4.75893706448089


In [ ]:
#convertir a float
x = x.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x /= np.ceil(np.max(x))

In [ ]:
print(np.max(x))
print(np.min(x))
print(np.mean(x))

0.99326605
0.00078926043
0.09517873


In [ ]:
#convertir de 3D a 4D
x = x.reshape((x.shape[0], x.shape[1], x.shape[2], 1))

print(x.shape)

(576, 264, 62, 1)


In [ ]:
def CNN_2D(n_filtros, tamano_filtros, n_neuronas):
  model = Sequential()
  model.add(Conv2D(n_filtros, tamano_filtros, activation='relu',
                   input_shape=x.shape[1:]))
  model.add(MaxPool2D((2,2)))
  model.add(Conv2D(n_filtros, tamano_filtros, activation='relu'))
  model.add(MaxPool2D((2,2)))
  model.add(Conv2D(n_filtros, tamano_filtros, activation='relu'))
  model.add(MaxPool2D((2,2)))
  model.add(Flatten())
  model.add(Dense(n_neuronas, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(n_classes, activation='softmax'))

  optimizer = Adam(lr=1e-4)
  model.compile(optimizer = optimizer,
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [ ]:
clf = KerasClassifier(build_fn = CNN_2D, epochs=400, batch_size=36, verbose=0)

In [ ]:
n_filtros = [4, 8]
tamano_filtros = [(3,3), (5,3), (25,3)]
n_neuronas = [16, 32]
hiperparametros = dict(n_filtros=n_filtros,
                       tamano_filtros=tamano_filtros,
                       n_neuronas=n_neuronas)

In [ ]:
# 12 redes = 
# x redes = 
2*3*2

12

In [ ]:
grid = GridSearchCV(clf, hiperparametros, cv=2)

In [ ]:
inicio = time.time()

grid.fit(x, y)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 619.2700507640839


In [ ]:
print(grid.best_params_)
print(grid.best_score_)

{'n_filtros': 8, 'n_neuronas': 32, 'tamano_filtros': (3, 3)}
0.5295138955116272


In [ ]:
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))

0.519 (+/-0.080) for {'n_filtros': 4, 'n_neuronas': 16, 'tamano_filtros': (3, 3)}
0.516 (+/-0.010) for {'n_filtros': 4, 'n_neuronas': 16, 'tamano_filtros': (5, 3)}
0.460 (+/-0.073) for {'n_filtros': 4, 'n_neuronas': 16, 'tamano_filtros': (25, 3)}
0.500 (+/-0.049) for {'n_filtros': 4, 'n_neuronas': 32, 'tamano_filtros': (3, 3)}
0.514 (+/-0.007) for {'n_filtros': 4, 'n_neuronas': 32, 'tamano_filtros': (5, 3)}
0.465 (+/-0.056) for {'n_filtros': 4, 'n_neuronas': 32, 'tamano_filtros': (25, 3)}
0.509 (+/-0.052) for {'n_filtros': 8, 'n_neuronas': 16, 'tamano_filtros': (3, 3)}
0.512 (+/-0.066) for {'n_filtros': 8, 'n_neuronas': 16, 'tamano_filtros': (5, 3)}
0.484 (+/-0.059) for {'n_filtros': 8, 'n_neuronas': 16, 'tamano_filtros': (25, 3)}
0.530 (+/-0.024) for {'n_filtros': 8, 'n_neuronas': 32, 'tamano_filtros': (3, 3)}
0.528 (+/-0.076) for {'n_filtros': 8, 'n_neuronas': 32, 'tamano_filtros': (5, 3)}
0.431 (+/-0.035) for {'n_filtros': 8, 'n_neuronas': 32, 'tamano_filtros': (25, 3)}


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jun  6 22:50:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    54W / 250W |   8245MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  